# Домашнее задание №4. Матричная факторизация

## Задачи - 25 баллов (+5 доп баллов)
1. Ноутбук `mf.ipynb` - 20 баллов
- ImplicitALS - 4 балла
- SVD - 4 балла
- Dataset with features - 2 балла
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.

Используемые библиотеки в рамках ДЗ
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```

## Импорты и данные

In [1]:
import os
import threadpoolctl
import requests

import numpy as np
import pandas as pd
import zipfile as zf

from tqdm.auto import tqdm
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")


import os
import os.path

import datetime
from datetime import timedelta
import joblib
from itertools import product
from collections import defaultdict
import dill

import warnings
warnings.simplefilter("ignore")

Если вдруг у вас нет данных, то используйте закомментированный код

In [2]:
# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [3]:
data_path = os.environ.get("DATA_PATH")
if data_path is None:
    data_path ="data_original"
    
interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

print(interactions.shape)
interactions.head(5)

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [4]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()

# dataset = Dataset.construct(train)

C:\Users\sharthZ23\AppData\Local\Temp\ipykernel_24360\3775397126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)


In [5]:
K_RECOS = 10
map10 = MAP(k=K_RECOS)

## ImplicitALS

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях. 

Правила пари: 
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [6]:
# train preprocessing...
train2 = train.copy()
### BEGIN SOLUTION
train2[Columns.Weight] = np.where(train2['watched_pct'] > 10, 3, 1)
config = {
    'cls': 'ImplicitALSWrapperModel',
    'model': {
        'factors': 32,
        'alpha': 10,
        'regularization': 0.5,
        'num_threads': 16,
        'random_state': 23
    }
}
### END SOLUTION
dataset = Dataset.construct(train2)

In [7]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'
config["verbose"] = 0

model = model_from_config(config)
model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.052

0.05425716030240344
CPU times: total: 11min 56s
Wall time: 57.2 s


## SVD

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид. 

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [8]:
# train preprocessing...
### BEGIN SOLUTION
config = {
    'cls': 'PureSVDModel',
    'factors': 2,
    'random_state': 23
}
dataset2 = Dataset.construct(train2)
### END SOLUTION

In [9]:
%%time
assert config['cls'] == 'PureSVDModel'
config["verbose"] = 0

model = model_from_config(config)
model.fit(dataset2)

recos = model.recommend(
    users=hot_users,
    dataset=dataset2,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.066

0.07157194088545771
CPU times: total: 19.4 s
Wall time: 13.6 s


## Dataset with features

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [10]:
### BEGIN SOLUTION 
users.fillna('Unknown', inplace=True)
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)

items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features = pd.concat((genre_feature, content_feature))
### END SOLUTION

dataset_with_features = Dataset.construct(
    interactions_df=train2,
### BEGIN SOLUTION   
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
### END SOLUTION
)

In [11]:
assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features

Собрав датасет с фичами вы готовы к третьему раунду пари. 

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [12]:
### BEGIN SOLUTION
config = {
    'cls': 'ImplicitALSWrapperModel',
    'model': {
        'factors': 128,
        'alpha': 10,
        'regularization': 0.5,
        'num_threads': 12,
        'random_state': 23
    },
    'fit_features_together': True,
}
### END SOLUTION

In [13]:
%%time
assert config['cls'] == 'ImplicitALSWrapperModel'
config["verbose"] = 0

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.073

E:\Anaconda3\envs\itmo_l4\Lib\site-packages\rectools\dataset\features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


0.07316924726612141
CPU times: total: 9min 36s
Wall time: 1min 11s


## LightFM with features

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего старшего коллегу, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [14]:
### BEGIN SOLUTION
config = {
    'cls': 'LightFMWrapperModel',
    'model': {
        'loss': 'warp',
        'no_components': 12,
        #'learning_rate': 0.05,
        #'user_alpha': 0.1,
        #'item_alpha': 0.1,
        'random_state': 23
    },
    'epochs': 10,
}
### END SOLUTION

In [ ]:
%%time
assert config['cls'] == 'LightFMWrapperModel'
config["verbose"] = 0

model = model_from_config(config)
model.fit(dataset_with_features)

recos = model.recommend(
    users=hot_users,
    dataset=dataset_with_features,
    k=K_RECOS,
    filter_viewed=True,
)
print(map10.calc(recos, test))

assert map10.calc(recos, test) >= 0.08

## Сервис

Эта битва была легендарной, но вот за спиной опять появляется ваш продакт. 

"Время катить АБ" - говорит он в пятницу вечером. Делать нечего, вы собираете ваши наработки и определяете совместный фронт работ.

В прод должна заехать лучшая модель, которая побьет текущей модели в проде в `MAP@10 = 0.075`.

Также есть бонус от вашего старшего коллеги, который вам советует присмотреться к `Approximate nearest neighbours`, например к `nmslib`.

"Если сможешь обернуть в ANN, то на следующем годовом ревью получишь от меня оценку отлично" - сказал он. Изучить новые технологии и получить повышение - идеально, заключаете вы и бросаетесь в бой.

# Вместо заключения

## Задачи - 25 баллов (30 баллов с доп задачей по ANN)
1. Ноутбук `mf.ipynb` - 20 баллов
- SVD - 5 баллов
- ImplicitALS - 5 баллов
- ImplicitALS with features - 5 баллов
- LightFM with features - 5 баллов
2. Имплементация модели в сервис - 5 баллов
- Пробить на Leaderboard порог `map@10 = 0.075`
- Если при этом используете MF (Implicit или LightFM) + ANN (nmslib, faiss, annoy и тд) - дополнительно 5 баллов
  
## Как сдать ноутбук `mf.ipynb` на проверку

1. Прогоните весь код ноутбука - проверьте, что нет ошибок и тесты проходят
2. Выложите готовый ноутбук в ваш репозиторий с сервисом из домашнего задания №1 по пути `notebooks/hw_4/mf.ipynb` в ветке `hw_4`
3. Проверьте, что есть доступ к вашему репозиторию для аккаунтов `https://github.com/feldlime`
4. Откройте PR в main ветку и добавьте в ревьюеры **своего ментора**
5. Не проводите мердж в `main` ветку, пока не увидите оценку за это ДЗ в ведомости. Файл с ноутбуком должен находиться в ветке `hw_4`

Обратите внимание, что сборка ноутбуков на проверку автоматизирована. В случае неправильного пути, имени файла или ветки (а также при отсутствии доступа у `@feldlime`) ваша работа не попадёт на проверку и получит `0` баллов.